In [129]:
import codecs
import pandas as pd
df = pd.read_csv("/content/judge-1377884607_tweet_product_company.csv",encoding="utf8",engine="python",encoding_errors="ignore")


In [130]:
df.head()

,tweet_text,emotion_in_tweet_is_directed_at,is_there_an_emotion_directed_at_a_brand_or_product
0,.@wesley83 I have a 3G iPhone. After 3 hrs twe...,iPhone,Negative emotion
1,@jessedee Know about @fludapp ? Awesome iPad/i...,iPad or iPhone App,Positive emotion
2,@swonderlin Can not wait for #iPad 2 also. The...,iPad,Positive emotion
3,@sxsw I hope this year's festival isn't as cra...,iPad or iPhone App,Negative emotion
4,@sxtxstate great stuff on Fri #SXSW: Marissa M...,Google,Positive emotion


In [131]:
df.shape

(9093, 3)

In [132]:
df.rename(columns={'emotion_in_tweet_is_directed_at':'emotion_towards','is_there_an_emotion_directed_at_a_brand_or_product':'emotion_reaction'},inplace=True)

In [133]:
df.head()

,tweet_text,emotion_towards,emotion_reaction
0,.@wesley83 I have a 3G iPhone. After 3 hrs twe...,iPhone,Negative emotion
1,@jessedee Know about @fludapp ? Awesome iPad/i...,iPad or iPhone App,Positive emotion
2,@swonderlin Can not wait for #iPad 2 also. The...,iPad,Positive emotion
3,@sxsw I hope this year's festival isn't as cra...,iPad or iPhone App,Negative emotion
4,@sxtxstate great stuff on Fri #SXSW: Marissa M...,Google,Positive emotion


In [134]:
df.emotion_reaction.value_counts()


No emotion toward brand or product    5389
Positive emotion                      2978
Negative emotion                       570
I can't tell                           156
Name: emotion_reaction, dtype: int64

In [235]:
def custom_encoder(df):
  df.replace(to_replace=["No emotion toward brand or product"], value=0, inplace=True)
  df.replace(to_replace=["Positive emotion"], value=1, inplace=True)
  df.replace(to_replace=["Negative emotion"], value=2, inplace=True)
  df.replace(to_replace=["I can't tell"], value=3, inplace=True)
  return df

In [236]:
df.emotion_reaction = custom_encoder(df.emotion_reaction)
df.head()

,tweet_text,emotion_reaction
0,.@wesley83 I have a 3G iPhone. After 3 hrs twe...,2
1,@jessedee Know about @fludapp ? Awesome iPad/i...,1
2,@swonderlin Can not wait for #iPad 2 also. The...,1
3,@sxsw I hope this year's festival isn't as cra...,2
4,@sxtxstate great stuff on Fri #SXSW: Marissa M...,1


In [237]:
df.head()

,tweet_text,emotion_reaction
0,.@wesley83 I have a 3G iPhone. After 3 hrs twe...,2
1,@jessedee Know about @fludapp ? Awesome iPad/i...,1
2,@swonderlin Can not wait for #iPad 2 also. The...,1
3,@sxsw I hope this year's festival isn't as cra...,2
4,@sxtxstate great stuff on Fri #SXSW: Marissa M...,1


In [238]:
from keras.preprocessing import text
tokenizer = text.Tokenizer() 
tokenizer.fit_on_texts(list(df['tweet_text']))
tokenized_texts = tokenizer.texts_to_sequences(df['tweet_text'])


In [239]:
df["emotion_reaction"] 

0       2
1       1
2       1
3       2
4       1
       ..
9088    1
9089    0
9090    0
9091    0
9092    0
Name: emotion_reaction, Length: 9093, dtype: int64

In [240]:
tokenized_texts[0]

[5838,
 23,
 51,
 11,
 605,
 18,
 257,
 111,
 2574,
 633,
 6,
 1347,
 25,
 32,
 86,
 887,
 23,
 104,
 5,
 1106,
 2575,
 3942,
 6,
 1]

In [241]:
from keras.utils import pad_sequences
X = pad_sequences(tokenized_texts, maxlen=100)

In [242]:
print(X[0])

[   0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0 5838   23   51   11  605   18  257  111
 2574  633    6 1347   25   32   86  887   23  104    5 1106 2575 3942
    6    1]


In [243]:
len(tokenizer.word_index)

10051

In [244]:
import numpy as np

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, df['emotion_reaction'].values, test_size=0.2)




In [245]:
from keras.models import Sequential
from keras.layers import Dense, LSTM, Embedding,SimpleRNN,Dropout

In [250]:
model = Sequential()
model.add(Embedding(input_dim = len(tokenizer.word_index)+1, output_dim = 128,input_length=100))
model.add(SimpleRNN(10))
model.add(Dropout(0.5))
model.add(Dense(50, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))  

In [251]:
model.compile(loss='binary_crossentropy', 
              optimizer='adam', 
              metrics=['accuracy'])

In [252]:
model.summary()

Model: "sequential_14"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_14 (Embedding)    (None, 100, 128)          1286656   
                                                                 
 simple_rnn_14 (SimpleRNN)   (None, 10)                1390      
                                                                 
 dropout_28 (Dropout)        (None, 10)                0         
                                                                 
 dense_28 (Dense)            (None, 50)                550       
                                                                 
 dropout_29 (Dropout)        (None, 50)                0         
                                                                 
 dense_29 (Dense)            (None, 1)                 51        
                                                                 
Total params: 1,288,647
Trainable params: 1,288,647
N

In [253]:
import numpy as np
X_train = np.asarray(X_train).astype(np.float32)
y_train= np.asarray(y_train).astype(np.float32)
model.fit(X_train, y_train, epochs=10, validation_split=0.1)

Epoch 1/10
205/205 [==============================] - 10s 41ms/step - loss: 0.6911 - accuracy: 0.4580 - val_loss: 0.6686 - val_accuracy: 0.5151
Epoch 2/10
205/205 [==============================] - 8s 39ms/step - loss: 0.6174 - accuracy: 0.5629 - val_loss: 0.5592 - val_accuracy: 0.5783
Epoch 3/10
205/205 [==============================] - 8s 39ms/step - loss: 0.3694 - accuracy: 0.6699 - val_loss: 0.6019 - val_accuracy: 0.6099
Epoch 4/10
205/205 [==============================] - 8s 39ms/step - loss: -0.2802 - accuracy: 0.7389 - val_loss: 0.8634 - val_accuracy: 0.5989
Epoch 5/10
205/205 [==============================] - 11s 53ms/step - loss: -2.5247 - accuracy: 0.7621 - val_loss: 1.7696 - val_accuracy: 0.6003
Epoch 6/10
205/205 [==============================] - 8s 37ms/step - loss: -7.4013 - accuracy: 0.7576 - val_loss: 3.2808 - val_accuracy: 0.5865
Epoch 7/10
205/205 [==============================] - 8s 37ms/step - loss: -15.7649 - accuracy: 0.7660 - val_loss: 5.0834 - val_accuracy:

In [254]:
y_pred = model.predict(X_test)

57/57 [==============================] - 1s 8ms/step
